In [145]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import networkx as nx

from tqdm import tqdm
from termcolor import colored
import sys
import os
import pandas as pd
import json
import csv
from collections import defaultdict

In [ ]:
CLASSES_PATH = os.path.dirname(os.path.abspath('../Classes/'))
if not (CLASSES_PATH in sys.path):
    sys.path.append(CLASSES_PATH)
from Classes.Files_Handler_Class import Files_Handler
from Classes.K_Shell_Calculate_Class import K_Shell_Calculate
from Classes.Resd_Network_Infos_Class import Resd_Network_Infos
from Classes.SIR_Diffusion_Model_Class import SIR_Diffusion_Model
from Classes.Get_Past_Results_Class import Get_Past_Results
from Classes.Network_Infos_Writer_Class import Network_Infos_Writer
from Classes.Layers_Ranking_Class_Old import Layers_Ranking
from Classes.Network_Node_Centrality_Class_Old import Network_Node_Centrality
from Classes.Bcolors_Class import Bcolors as bcolors
from Classes.CSV_Files_Class import CSV_Files

In [147]:

seed_set_size = 5
version_num = 'v02'

color_list = ["light_red", "light_green", "light_yellow",
               "light_blue","light_magenta", "light_cyan",
               "blue", "red", "white", "green", "yellow",
                 "magenta", "cyan", ]
tqdm_color_list = ['blue', 'red', 'green', 'cyan', 'magenta', 'yellow', 'black', 'white']

In [148]:
layer_centrality = ['layer_density','layer_degree_histogram','layer_edge_weight',
                    'layer_sombor_index', 'layer_nodes_weight','layer_k_shell_weight']
node_centrality = ['degree', 'clustering', 'nip', 'sombor_index', 'ego_density','ego_degree',
                     'ego_k_shell', 'ego_degree_mean','kss', 'vote_power']
drop_centrality = ['layer_id', 'node_id', 'weight', 'k_shell', 'k_shell_itr']

In [149]:
source_code_path = str(os.getcwd())
source_code_path = source_code_path.replace("\\", "/")

In [150]:
csv_files_obj = CSV_Files()
files_handler_obj = Files_Handler()
file_path = files_handler_obj.select_files("text files", ".edgeslist .edgelist .edges .mtx .txt", 'Select Dataset')
if file_path is None or file_path == '':
    sys.exit("File Selection Canceled !")
file_info = files_handler_obj.get_file_path_info(file_path)
network_name = file_info['name']
network_type = file_info['type']
network_path = file_info['path']
if network_name == "":
    sys.exit("Dont Network Selection!")
file_info


{'path': 'D:/Masters thesis/Methods For Compare/Temp/12 - realitycommons (84 Node and 5 Layer)/',
 'name': 'Relationshipsfromsurveys',
 'type': '.edgelist'}

In [151]:
resd_network_infos_object = Resd_Network_Infos()
(
    network_layers_info,
    network_layers_nodes,
    entra_layer_edges,
    entra_layer_edges_features,
    inter_layer_edge,
) = Resd_Network_Infos.read_nodeFrom_layerFrom_nodeTo_layerTo(
    network_path, network_name, network_type
)

In [152]:
print(file_info['name'], '\n')
layers_id = []
network_layers_count = len(network_layers_info)
graphs_of_network = [None] * network_layers_count
network_entier_edges = ""
layers_nodes_infect_scale = []

i = 0
j = 0
while i < network_layers_count:
    graphs_of_network[i] = nx.Graph()
    network_layers_nodes[i] = list(set(network_layers_nodes[i]))
    layers_nodes_infect_scale.append({})
    if len(network_layers_nodes[i]) > 0:
        graphs_of_network[i].add_edges_from(entra_layer_edges[i])

        nx.set_node_attributes(graphs_of_network[i], None, "degree")
        nx.set_node_attributes(graphs_of_network[i], None, "k_shell")
        nx.set_node_attributes(graphs_of_network[i], None, "k_shell_itr")
        nx.set_node_attributes(graphs_of_network[i], None, "nip")
        nx.set_node_attributes(graphs_of_network[i], None, "sombor_index")
        nx.set_node_attributes(graphs_of_network[i], None, "ego_density")
        nx.set_node_attributes(graphs_of_network[i], None, "ego_degree")
        nx.set_node_attributes(graphs_of_network[i], None, "ego_k_shell")
        nx.set_node_attributes(graphs_of_network[i], None, "ego_degree_mean")
        nx.set_node_attributes(graphs_of_network[i], None, "kss")
        nx.set_node_attributes(graphs_of_network[i], 1, "vote_power")
        nx.set_node_attributes(graphs_of_network[i], None, "clustering")
        nx.set_node_attributes(graphs_of_network[i], None, "SIR")
        
        graphs_of_network[i].graph["id"] = i
        graphs_of_network[i].graph["layer_density"] = nx.density(graphs_of_network[i])
        graphs_of_network[i].graph["layer_degree_histogram"] = None
        graphs_of_network[i].graph["layer_edge_weight"] = None
        graphs_of_network[i].graph["layer_sombor_index"] = None
        graphs_of_network[i].graph["layer_nodes_weight"] = None
        graphs_of_network[i].graph["layer_k_shell_weight"] = None

        K_Shell_Calculate_Object = K_Shell_Calculate(graphs_of_network[i])
        graphs_of_network[i] = K_Shell_Calculate_Object.get_k_shell_info()
        del K_Shell_Calculate_Object
        degrees = dict(graphs_of_network[i].degree())
        nx.set_node_attributes(graphs_of_network[i], degrees, "degree")
        layers_id.append(str(i))
        print(colored("Layer "  + str(i) + ": " + str(graphs_of_network[i].number_of_nodes()) + " Node And " +
                       str(graphs_of_network[i].number_of_edges()) + " Edge", color_list[j]))
        # print(colored(graphs_of_network[i].graph['k_shell_info'], color_list[i]))
    i += 1
    j += 1
    if j >= len(color_list):
        j = 0

network_entier_nodes_list = []
for item in network_layers_nodes:
    network_entier_nodes_list += item

network_entier_nodes_list = list(set(network_entier_nodes_list))
network_entier_nodes_count = len(network_entier_nodes_list)
print()
print("network entier nodes : " + colored(str(network_entier_nodes_count), "yellow"))

Relationshipsfromsurveys 

Layer 1: 82 Node And 620 Edge
Layer 2: 84 Node And 1327 Edge
Layer 3: 84 Node And 1237 Edge
Layer 4: 83 Node And 2132 Edge
Layer 5: 83 Node And 1956 Edge

network entier nodes : 84


In [153]:
get_past_results_obj = Get_Past_Results(network_path, network_name)
network_infos_writer_object = Network_Infos_Writer(network_path, network_name)

In [154]:
our_model_path = files_handler_obj.make_dir(file_info['path'], 'Our Model')

In [155]:
del network_layers_nodes, entra_layer_edges
del entra_layer_edges_features

In [156]:
print(network_name)
layer_past_result_status = False
layer_past_result_status = get_past_results_obj.get_past_layer_att_results(graphs_of_network)
if layer_past_result_status:
        print("Load layer results: ", bcolors.HEADER + bcolors.OKGREEN + str(layer_past_result_status) + bcolors.ENDC)
else:
        print("Load layer results: ", bcolors.HEADER + bcolors.FAIL + str(layer_past_result_status) + bcolors.ENDC)
if not layer_past_result_status:
        layers_ranking_object = Layers_Ranking()
        print(bcolors.HEADER + bcolors.OKBLUE + 'Calc layer layers_density_weight' + bcolors.ENDC)
        layers_ranking_object.layers_density_weight(graphs_of_network)
        print(bcolors.HEADER + bcolors.OKBLUE + '\nCalc layer layers_degree_distribution' + bcolors.ENDC)
        layers_ranking_object.layers_degree_distribution(graphs_of_network)
        print(bcolors.HEADER + bcolors.OKBLUE + '\nCalc layer layers_edges_and_sombor_index' + bcolors.ENDC)
        layers_ranking_object.layers_edges_and_sombor_index(graphs_of_network)
        print(bcolors.HEADER + bcolors.OKBLUE + '\nCalc layer layers_nodes_weight' + bcolors.ENDC)
        layers_ranking_object.layers_nodes_weight(graphs_of_network)
        print(bcolors.HEADER + bcolors.OKBLUE + '\nCalc layer layers_k_shell_weight' + bcolors.ENDC)
        layers_ranking_object.layers_k_shell_weight(graphs_of_network)

Relationshipsfromsurveys
Load layer results:  True


In [157]:
if not layer_past_result_status:
    network_infos_writer_object.write_network_layer_infos_csv(graphs_of_network)

In [158]:
past_node_att_file = get_past_results_obj.get_past_node_att_file()
if past_node_att_file != False:
    node_past_result_att_status = float(past_node_att_file.split('/')[-1].split(' ')[-2].split('=')[-1])
else:
    node_past_result_att_status = 0
print(node_past_result_att_status)


1.0


In [159]:
print('Layer\t density\t degree_histogram\t edge_weight\t sombor_index\t nodes_weight\t k_shell_weight')
f_p = '{:9.7f}' # float padding
c = 0
for i, graph in enumerate(graphs_of_network):
    if graph.number_of_nodes() > 0:
        print(colored((str(i)+ ' \t ' + str(f_p.format(graph.graph["layer_density"]))+ ' \t ' +
                       str(f_p.format(graph.graph["layer_degree_histogram"]))+ ' \t\t ' +
                       str(f_p.format(graph.graph["layer_edge_weight"]))+ ' \t ' +
                       str(f_p.format(graph.graph["layer_sombor_index"]))+ ' \t ' +
                       str(f_p.format(graph.graph["layer_nodes_weight"]))+ ' \t ' +
                       str(f_p.format(graph.graph["layer_k_shell_weight"]))),
                       color_list[c]))
    c += 1
    if c >= len(color_list):
        c = 0

Layer	 density	 degree_histogram	 edge_weight	 sombor_index	 nodes_weight	 k_shell_weight
1 	 0.1866908 	 15.1219512 		 0.0062000 	 3.3709498 	 0.0082000 	 30.5788569
2 	 0.3806655 	 31.5952381 		 0.0132700 	 5.4945591 	 0.0084000 	 54.1307046
3 	 0.3548480 	 29.4523810 		 0.0123700 	 5.1603057 	 0.0084000 	 83.0354331
4 	 0.6265060 	 51.3734940 		 0.0213200 	 7.8326236 	 0.0083000 	 171.7824985
5 	 0.5747870 	 47.1325301 		 0.0195600 	 7.4284391 	 0.0083000 	 240.3359174


In [160]:
print(network_name)
# past_results_status = False
past_results_status, past_results_nodes = get_past_results_obj.get_past_node_att_results(graphs_of_network)
if past_results_status:
    print("\nLoad temp results: ", bcolors.HEADER + bcolors.OKGREEN + str(past_results_status) + bcolors.ENDC)
    print("Load temp results count: ", bcolors.HEADER + bcolors.OKGREEN + str(len(past_results_nodes)) + bcolors.ENDC, '\n')
else:
    print("Load temp results: ", bcolors.HEADER + bcolors.FAIL + str(past_results_status) + bcolors.ENDC, '\n')
Network_Node_Centrality_obj = Network_Node_Centrality()
hop_num = 2
Network_Node_Centrality_obj.get_nodes_centrality(network_infos_writer_object, graphs_of_network, None, hop=hop_num)
network_infos_writer_object.write_network_nodes_info_csv(1, graphs_of_network)


Relationshipsfromsurveys

Load temp results:  True
Load temp results count:  84 


Layer 1: Graph with 82 nodes and 620 edges


100%|██████████| 82/82 [00:00<00:00, 82084.23 Node/s]



Layer 2: Graph with 84 nodes and 1327 edges


100%|██████████| 84/84 [00:00<00:00, 83986.06 Node/s]



Layer 3: Graph with 84 nodes and 1237 edges


100%|██████████| 84/84 [00:00<?, ? Node/s]


Layer 4: Graph with 83 nodes and 2132 edges



100%|██████████| 83/83 [00:00<00:00, 83144.79 Node/s]



Layer 5: Graph with 83 nodes and 1956 edges


100%|██████████| 83/83 [00:00<?, ? Node/s]


In [161]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [162]:
class Multilayer_Full_Model(nn.Module):

    def __init__(
        self, node_in_features: int, node_out_features: int,
        layer_in_features: int, layer_out_features: int,
        encoder_head: int, num_encoder:int, encoder_activation: str,
        bias: bool, dropout: float,
        activation: nn.modules.activation, device: str = "cpu",
        h0:int = 8, h1: int = 16, h2: int = 32, h3: int = 64, h4: int = 128, h5 = 256
    ):
        super().__init__()
        self.node_embeding = nn.Sequential(
            nn.Linear(in_features=node_in_features, out_features=h1, bias=bias, device=device),
            activation,
            nn.Linear(in_features=h1, out_features=h2, bias=bias, device=device),
            activation,
            nn.Linear(in_features=h2, out_features=h3, bias=bias, device=device),
            activation,
            nn.Linear(in_features=h3, out_features=h4, bias=bias, device=device),
            activation,
            nn.Linear(in_features=h4, out_features=node_out_features, bias=bias, device=device)
        )
        self.layer_embeding = nn.Sequential(
            nn.Linear(in_features=layer_in_features, out_features=h1, bias=bias, device=device),
            activation,
            nn.Linear(in_features=h1, out_features=h2, bias=bias, device=device),
            activation,
            nn.Linear(in_features=h2, out_features=h3, bias=bias, device=device),
            activation,
            nn.Linear(in_features=h3, out_features=h4, bias=bias, device=device),
            activation,
            nn.Linear(in_features=h4, out_features=node_out_features, bias=bias, device=device)
            )
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=node_out_features, nhead=encoder_head,
                dim_feedforward=(4 * node_out_features), dropout=dropout,
                activation=encoder_activation, bias=bias,
                batch_first=True, device=device),
            num_encoder)
        self.regression = nn.Sequential(
            nn.Linear(in_features=node_out_features, out_features=h4, bias=bias, device=device),
            activation,
            nn.Linear(in_features=h4, out_features=h3, bias=bias, device=device),
            activation,
            nn.Linear(in_features=h3, out_features=h2, bias=bias, device=device),
            activation,
            nn.Linear(in_features=h2, out_features=h1, bias=bias, device=device),
            activation,
            nn.Linear(in_features=h1, out_features=h0, bias=bias, device=device),
            activation,
            nn.Linear(in_features=h0, out_features=1, bias=bias, device=device),

        )

    def forward(self, node_x, layer_x):
        node_y = self.node_embeding(node_x).unsqueeze(dim=2)
        layer_y = self.layer_embeding(layer_x).unsqueeze(dim=1)
        y = torch.matmul(node_y, layer_y)
        y = self.encoder(y)
        y = torch.mean(y, dim=1)
        y = self.regression(y)
        return y

In [163]:
model_file = ''
files_names = files_handler_obj.get_files_in_path(source_code_path)
for item in files_names:
    if item.split('.')[-1] == 'pt':
        model_file = item
        break
model_file

'best_loss_valid model lr=0.0001 wd=1e-05 epochs=825  loss_valid=2.8144 loss_train=2.8536.pt'

In [164]:
model_path = source_code_path + '/' +model_file
model = None
try:
    model = torch.load(model_path, weights_only= False, map_location=torch.device(device))
    torch.set_grad_enabled(False)
    print(bcolors.OKGREEN + 'Model load' + bcolors.ENDC)
except Exception as e:
    print(bcolors.FAIL + 'Model not found!' + bcolors.ENDC)
    sys.exit(e)

Model load


In [165]:
import csv
import pandas as pd

network_nodes_list_SIR_p = {}
nodes_SIR_p = defaultdict(dict)
SIR_p_file_status = False
SIR_p_file = file_info['path'] + file_info['name'] + ' SIR_p/' + file_info['name'] + ' SIR_p.csv'
try:
    with open(SIR_p_file) as csv_file:
        reader = csv.reader(csv_file)
        network_nodes_list_SIR_p = dict(reader)
    SIR_p_file_status = True
except:
    SIR_p_file_status = False

nodes_SIR_p_file_status = False
nodes_SIR_p_file = file_info['path'] + file_info['name'] + ' SIR_p/' + file_info['name'] + ' temp SIR_p.csv'
try:
    nodes_SIR_p = pd.read_csv(nodes_SIR_p_file, index_col=0)
    nodes_SIR_p = pd.DataFrame.to_dict(nodes_SIR_p, orient="index")
    nodes_SIR_p_file_status = True
except Exception as e:
    nodes_SIR_p_file_status = False

print(SIR_p_file_status, nodes_SIR_p_file_status)



True True


In [166]:
if nodes_SIR_p_file_status:
    for graph in graphs_of_network:
        if graph.number_of_nodes() > 0 and graph.number_of_edges() > 0:
            nx.set_node_attributes(graph, None, "SIR_p")
    for node in network_entier_nodes_list:
        node_SIR_p_values = nodes_SIR_p[int(node)]
        for k, v in node_SIR_p_values.items():
            if k != 'AVG':
                if node in graphs_of_network[int(k)].nodes():
                    graphs_of_network[int(k)].nodes[node]['SIR_p']= float(v)

In [167]:
def get_model_outputs(model, node_data_loader:DataLoader, layer_data:torch.Tensor, device):
  model.eval()
  outputs = []
  with tqdm(node_data_loader, unit=" batch") as tepoch:
    with torch.no_grad():
      for i, (node_inputs, _) in enumerate(tepoch):
        tepoch.set_description(f"Batch {i + 1}")
        node_inputs = node_inputs.to(device)
        layer_inputs = layer_data.to(device)
        outputs += (((model(node_inputs, layer_inputs)).squeeze(dim=1)).tolist())
  return outputs

In [168]:
node_mu = torch.FloatTensor([ 26.0719,   0.1327,  19.4767,  12.0195,   3.8776, -14.1054, -18.9498, 31.2976,   2.8534,   8.7193])
node_std = torch.FloatTensor([ 78.4243,   0.2285,  89.8148,  50.2024,  27.2598,  81.4585, 189.8591, 115.6929,  10.7834,  60.4080])

layer_mu = torch.FloatTensor([3.6536e-03, 1.9344e+01, 4.9918e+04, 2.9013e+02, 3.5260e+04, 6.3725e+01])
layer_std = torch.FloatTensor([2.1411e-02, 2.6726e+01, 1.4293e+05, 1.3756e+03, 9.9566e+04, 1.5917e+02])

node_scale_vector = torch.FloatTensor([10000, 1000000, 1000000000, 100000, 1000000000, 1000000000, 1000000, 100000, 10000, 1])
layer_scale_vector = torch.FloatTensor([1, 10, 10, 100, 1, 100])

In [169]:
real_sir = []
model_sir = []

In [ ]:
batch_size = 32
layers_counter = len(graphs_of_network)
if not nodes_SIR_p_file_status and len(network_entier_nodes_list) != len(network_nodes_list_SIR_p.keys()):
    tqdm_color_index = 0
    for graph in graphs_of_network:
        if graph.number_of_nodes() > 0 and graph.number_of_edges() > 0:
            degrees = dict(graph.degree())  # Get the degree of each node
            mean_degree = sum(degrees.values()) / len(degrees)
            print(bcolors.FAIL + f'\nlayer {layers_counter}: {graph}' + bcolors.ENDC)
            real_sir.append([])
            model_sir.append([])
        # ---------- Extract layer features ---------------------
            layer_x_data = []
            layer_info = graph.graph
            layer_id = layer_info['id']
            for k, v in layer_info.items():
                if (k in layer_centrality):
                        layer_x_data.append(v)
            
            layer_x_data = torch.FloatTensor(layer_x_data)
            layer_x_data = layer_x_data / layer_scale_vector
            layer_x_data = (layer_x_data - layer_mu) / layer_std
            layer_x_data = layer_x_data.to(device)
        # -------------------------------------------------------
        # ---------- Extract nodes features ---------------------
            nodes_list = list(graph.nodes())
            with tqdm(nodes_list, unit=" Node") as tepoch:
                if tqdm_color_index >= len(tqdm_color_list):
                    tqdm_color_index = 0
                tepoch.colour = tqdm_color_list[tqdm_color_index]
                tqdm_color_index += 1
                for i, node in enumerate(tepoch):
                    tepoch.set_description(f"Node {i + 1}")
                    node_x_data = []
                    node_info = graph.nodes[node]
                    for centrality in node_centrality:
                        node_x_data.append(float(node_info[centrality]))
            # -------------------------------------------------------
                    node_x_data = torch.FloatTensor(node_x_data)
                    node_x_data = node_x_data / node_scale_vector
                    node_x_data = (node_x_data - node_mu) / node_std
                    node_x_data = node_x_data.to(device)
                # -----Feeding data to neural network and get outputs----
                    with torch.no_grad():
                        output = model(node_x_data.unsqueeze(0), layer_x_data.unsqueeze(0)).squeeze(1).tolist()[0]
                    
                    output = output * (graphs_of_network[layer_id].degree(node) / mean_degree)
                    graphs_of_network[layer_id].nodes[node]["SIR_p"] = output
                    nodes_SIR_p[node][layer_id] = output
                    real_sir[-1].append(graphs_of_network[layer_id].nodes[node]["SIR"])
                    model_sir[-1].append(output)
        # -------------------------------------------------------
        layers_counter -= 1
else:
    for graph in graphs_of_network:
        if graph.number_of_nodes() > 0 and graph.number_of_edges() > 0:
            real_sir.append([])
            model_sir.append([])
            layer_id = graph.graph['id']
            nodes_list = list(graph.nodes())
            for node in nodes_list:
                if not(graphs_of_network[layer_id].nodes[node]["SIR"] is None):
                    real_sir[-1].append(graphs_of_network[layer_id].nodes[node]["SIR"])
                    model_sir[-1].append(graphs_of_network[layer_id].nodes[node]["SIR_p"])

print(bcolors.OKGREEN + '\nDone.' + bcolors.ENDC)


Done.


In [171]:
if not nodes_SIR_p_file_status:
    path_SIR_p =  files_handler_obj.make_dir(file_info['path'], file_info['name']+' SIR_p')
    write_layers_SIR_p_status = network_infos_writer_object.write_layers_SIR_p_to_csv(path_SIR_p, file_info['name']+" temp SIR_p", nodes_SIR_p)
    if write_layers_SIR_p_status == 'Done.':
        print(bcolors.OKGREEN + write_layers_SIR_p_status + bcolors.ENDC)

In [172]:
# loss_fn = nn.HuberLoss()
# for i, item in enumerate(model_sir):
#     loss = loss_fn(torch.FloatTensor(real_sir[i]), torch.FloatTensor(model_sir[i]))
#     print(f"Layer {i + 1}: {loss.tolist()}")

In [173]:
node_by_node_SIR_p_writer_status = SIR_p_file_status
SIR_p_opened_file = None
print(len(network_nodes_list_SIR_p.keys()))
node_SIR_p_write_counter = 0
nodes_SIR_p_value = {}
if len(network_entier_nodes_list) != len(network_nodes_list_SIR_p.keys()):
    node_counter = len(network_nodes_list_SIR_p.keys())
    print(bcolors.FAIL + 'calculating global SIR_p:' + bcolors.ENDC)
    with tqdm(network_entier_nodes_list, unit=" Node") as t_nodes:
        for node in t_nodes:
            if node not in network_nodes_list_SIR_p:
                t_nodes.set_description(f"Node {node_counter}")
                node_counter += 1
                node_SIR_p = {}
                for graph in graphs_of_network:
                    if graph.number_of_nodes() > 0 and graph.number_of_edges() > 0 and node in graph.nodes():
                        layer_id = graph.graph['id']
                        node_SIR_p[layer_id] = float(graph.nodes[node]['SIR_p'])
                node_SIR_p = {k: v for k, v in sorted(node_SIR_p.items(),key=lambda item: item[1], reverse=True)}
                SIR_p = None
                common_neighbors = set()
                for i, (k_layer_id, v) in enumerate(node_SIR_p.items()):
                    if SIR_p is None:
                        SIR_p = v
                        common_neighbors = set(list(graphs_of_network[k_layer_id].neighbors(node)))
                    else:
                        node_neighbors = set(list(graphs_of_network[k_layer_id].neighbors(node)))
                        # node_neighborhood_effect = len(node_neighbors - common_neighbors) len(node_neighbors)
                        node_neighborhood_effect = (len(node_neighbors) - len(node_neighbors - common_neighbors)) * (i * 0.01)
                        common_neighbors = set.union(common_neighbors, node_neighbors)
                        # if node_neighborhood_effect < 0.25:
                        #     node_neighborhood_effect = 0.25
                        SIR_p += (v - node_neighborhood_effect)
                network_nodes_list_SIR_p[node] = SIR_p
                nodes_SIR_p_value[node] = SIR_p
                node_SIR_p_write_counter += 1
                if node_SIR_p_write_counter >= 1000 or len(network_nodes_list_SIR_p.keys()) == network_entier_nodes_count:
                    node_by_node_SIR_p_writer_status = network_infos_writer_object.node_SIR_p_writer(node_by_node_SIR_p_writer_status, SIR_p_file, nodes_SIR_p_value)
                    nodes_SIR_p_value = {}
                    node_SIR_p_write_counter = 0
    network_nodes_list_SIR_p
if not SIR_p_opened_file is None:
    SIR_p_opened_file.close()

84


In [174]:
if not SIR_p_file_status:
    network_nodes_list_SIR_p = {k: v for k, v in sorted(network_nodes_list_SIR_p.items(),
                                                          key=lambda item: item[1], reverse=True)}
# network_nodes_list_SIR_p


In [175]:
if not SIR_p_file_status:
   import pandas as pd
   SIR_p_file_root = files_handler_obj.make_dir(file_info['path'], file_info['name'] + ' SIR_p')
   SIR_p_file_info = SIR_p_file_root + file_info['name'] + ' SIR_p.csv'
   (pd.DataFrame.from_dict(data=network_nodes_list_SIR_p, orient='index')
      .to_csv(SIR_p_file_info, header=False))

In [176]:
network_nodes_list_SIR_p = list(network_nodes_list_SIR_p.items())

In [177]:
# network_nodes_list_SIR_p

In [178]:
print(f"Seed set size: {seed_set_size}")
i = 0
seed_set = []
for candidate_node, SIR_p in network_nodes_list_SIR_p:
    if len(seed_set) < seed_set_size:
        if i == 0:
            seed_set.append(candidate_node)
        else:
            # seed_set.append(candidate_node)
            common_neighbors_len = 0
            node_neighbors_len = 0
            for seed_node in seed_set:
                for graph in graphs_of_network:
                    if seed_node in graph and candidate_node in graph:
                        common_neighbors_len += len(nx.common_neighbors(graph, seed_node, candidate_node))
                        node_neighbors_len += graph.degree[candidate_node]
            if node_neighbors_len > 0:
                common_neighbors_percentage = common_neighbors_len / node_neighbors_len
            else: 
                common_neighbors_percentage = 0
            beta_scale = i * 0.01
            if beta_scale > 0.5:
                beta_scale = 0.5
            candidate_node_efficient_SIR_p = float(SIR_p) - ((float(SIR_p) * common_neighbors_percentage) * beta_scale)
            # print(SIR_p, common_neighbors_percentage, '\n', beta_scale, candidate_node_efficient_SIR_p, '\n')
            if i < (len(network_nodes_list_SIR_p) - 1):
                if candidate_node_efficient_SIR_p >= float(network_nodes_list_SIR_p[i + 1][1]):
                    seed_set.append(candidate_node)
            else:
                if candidate_node_efficient_SIR_p <= 0.25:
                    seed_set.append(candidate_node)
    else:
        break
    i += 1
    if i >= len(network_nodes_list_SIR_p):
        for candidate_node, SIR_p in network_nodes_list_SIR_p:
            if len(seed_set) < seed_set_size:
                if not candidate_node in seed_set:
                    seed_set.append(candidate_node)


seed_set

Seed set size: 5


['61', '3', '44', '8', '75']

In [179]:
print(f"Network name: {network_name}")
print(f"Seed set size: {seed_set_size}")
beta = 0.01
landa = 0.7
epoch = 2000
SIR_diffusion_model_obj = SIR_Diffusion_Model()
infection = SIR_diffusion_model_obj.synchronous_SIR_multilayer_with_seed_set_model(graphs_of_network, seed_set, beta, landa, epoch, network_entier_nodes_list)

sir_results_infos = {}
sir_results_infos['infection'] = infection
sir_results_infos['percentage'] = infection / network_entier_nodes_count
sir_results_infos['seed_set'] = seed_set

network_infos_writer_object.write_results_in_file(our_model_path, f'infection k={seed_set_size} beta={beta} landa={landa} epoch={10000}', sir_results_infos)
print(f"Network entier nodes count: {network_entier_nodes_count}")
print(f"Infected nodes count: {infection}")
print(f"Percentage of infection: {sir_results_infos['percentage']}")



Network name: Relationshipsfromsurveys
Seed set size: 5


Infect scale 0.85346: 100%|██████████| 2000/2000 [00:26<00:00, 76.18 Iteration/s]

Network entier nodes count: 84
Infected nodes count: 71.6905
Percentage of infection: 0.8534583333333333


In [180]:
# os.system('shutdown -s')